In [9]:
%pip install pypdf
%pip install llama-index

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from langchain.llms import LlamaCpp
from transformers import LlamaForCausalLM, LlamaTokenizer
from llama_index import VectorStoreIndex, SimpleDirectoryReader, LLMPredictor, PromptHelper, ServiceContext, StorageContext, load_index_from_storage


llm = LlamaCpp(model_path="./llama-2-13b-chat.Q4_K_M.gguf",
        n_gpu_layers=8192,
        n_batch=512,
        max_tokens=512,
        n_ctx=2048,
        f16_kv=True,
        verbose=True
)
llm_predictor = LLMPredictor(llm=llm)
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model='local')    

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA TITAN RTX, compute capability 7.5
llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from ./llama-2-13b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    6:     

In [2]:

# check if storage already exists
if not os.path.exists('./storage'):
    # load the documents and create the index
    documents = SimpleDirectoryReader("./main_data").load_data()
    index = VectorStoreIndex.from_documents(documents, service_context=service_context)
    # store it for later
    index.storage_context.persist()
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir="./storage")
    index = load_index_from_storage(storage_context, service_context=service_context)

In [4]:
query_engine = index.as_query_engine()
response = query_engine.query("What are the requirements for studying informatics in masters degree at the HTWG? Also list the sources from the documents")
print(response)

Llama.generate: prefix-match hit



The requirements to study informatics master's degree at HTWG are as follows:  
- The first degree should be in informatics or related fields such as business administration and economics. A minimum of 100 ECTS in informatics or related disciplines is required.  
Furthermore, students must have completed courses in the following areas: a) fundamentals of informatics, b) programming, c) algorithms and data structures, d) databases. 
Additionally, students must have passed modules from the following categories: a) basics of business administration, b) accounting, c) supply chain management, and d) controlling. 
A minimum grade point average of 2.4 is also required. The documents that provide information on these requirements are the "Hochschule Konstanz - Masterstudiengänge - Informatik" brochure and the "Zugangsvoraussetzungen BIT" document.



llama_print_timings:        load time =     387.01 ms
llama_print_timings:      sample time =      74.60 ms /   210 runs   (    0.36 ms per token,  2814.86 tokens per second)
llama_print_timings: prompt eval time =     590.51 ms /   652 tokens (    0.91 ms per token,  1104.14 tokens per second)
llama_print_timings:        eval time =    4278.84 ms /   209 runs   (   20.47 ms per token,    48.84 tokens per second)
llama_print_timings:       total time =    5275.33 ms
